In [47]:
import pandas as pd
import datetime
import math
from datetime import datetime, timedelta
import time

# define paths
data_path = '/Users/simonneumeyer/Dropbox/Ethiopia IE - Road Safety/Data/'
traffic_path = 'ETRE - Traffic/'
traffic_file = 'FinalData/traffic.pq'
segment_time_file = 'Time Segment Data/segment_time_panel.pq'

# choice of segment granularity BEFORE aggregation (for speeding up purposes) :
granularity_km_ex_ante = None
granularity_hour_ex_ante = None

# choice of segment granularity AT aggregation:
granularity_km_ex_post = None
granularity_hour_ex_post = None

# functions

def rounder(x, prec=2, base=.05):
    return round(base * round(float(x)/base),prec)

def datetime_range(start, end, delta):
    current = start
    while current <= end:
        yield current
        current += delta

def interpolate_row(row):
    """
    This function takes as input a traffic observation (a car entering & exiting the highway at a
    specific km and hour), interpolates the time at each kilometer with average speed
    and creates and returns a dataframe with all the inferred km-hour combinations as observations
    which can then easily be aggregated in the main function.
    :param row: a car entering & exiting the highway at a specific km and hour
    :return: dataframe with all the inferred km-hour combinations as observations
    """
    # preprocess:
    ent_date = datetime.strptime(str(row['ent_occur_time']), "%Y-%m-%d %H:%M:%S")
    exit_date = datetime.strptime(str(row['trans_occur_time']), "%Y-%m-%d %H:%M:%S")
    n_km = row['exit_km'] - row['entrance_km']

    # create seconds per kilometer variable to interpolate the time at each kilometer:
    speed_sec_km = (exit_date - ent_date).total_seconds() / abs(n_km)
    speed_sec_km = math.floor(speed_sec_km)

    new_index = [pd.to_datetime(dt.strftime("%Y-%m-%d %H:%M:%S")) for dt in
                 datetime_range(ent_date, exit_date, timedelta(seconds=speed_sec_km))]

    # define step direction (for when entrance_km > exit_km) for km_count:
    step_direction = int(n_km / abs(n_km))
    km_count = [j for j in range(int(row['entrance_km']), int(row['exit_km'] + step_direction), step_direction)]

    # create new dataframe to conduct aggregation on later:
    dt_one = pd.DataFrame(km_count, index=new_index[:len(km_count)], columns=['km'])
    dt_one['hour'] = dt_one.index.hour
    dt_one['year'] = dt_one.index.year
    dt_one['month'] = dt_one.index.month
    dt_one['day'] = dt_one.index.day
    dt_one['tstamp'] = pd.to_datetime(dt_one[['year', 'month', 'day']])
    dt_one['direction'] = row['direction']
    dt_one['cars'] = oversampling_factor * row['cars']
    dt_one['total_weight'] = oversampling_factor * row['total_weight']
    dt_one['speed_km_hr'] = oversampling_factor * row['speed_km_hr']
    dt_one['speed_sec_km'] = oversampling_factor * speed_sec_km
    
    for i in range(1,8):
        dt_one[f'veh_type_{i}'] = oversampling_factor * row[f'veh_type_{i}']
    
    
    dt_one_tr = dt_one.set_index(['tstamp', 'hour', 'km', 'direction'])[['cars', 'total_weight', 'speed_km_hr',
                                                                       'veh_type_1', 'veh_type_2', 'veh_type_3',
                                                                       'veh_type_4', 'veh_type_5', 'veh_type_6',
                                                                       'veh_type_7', 'speed_sec_km']]
    
    return dt_one_tr

def main(data, frac):
    """
    This function does 3 things:
    1. Performs, if needed, undersampling of the traffic file
    2. Performs the interpolate_row function on every row of the traffic dataset
        and concatenates the outputs.
    3. Aggregates the number of cars by hour & km to return the final traffic feature
    :param data: traffic data
    :param frac: fraction of random sample of traffic data
    :return: traffic feature dataframe
    """
    # Run over a random subset of the data:
    if frac is not None:
        data = data.sample(frac=frac)
        data = data.reset_index()
    data_dict = {}
    tic = time.time()
    for i in data.index:
        if i%2000==0:
            toc = time.time()
            print(f'{round(i/len(data)*100,2)}% done, {round(toc-tic,2)} seconds')
            tic = toc

        data_dict[i] = interpolate_row(data.loc[i, :])
        
    traffic = pd.concat(data_dict, axis=0, names=['car', 'tstamp', 'hour', 'km', 'direction'])
#    traffic = pd.concat(
#        {
#            i: interpolate_row(data.loc[i, :]) for i in data.index
#        }, axis=0, names=['car', 'tstamp', 'hour', 'km', 'direction'])

    # ex post aggregation if needed:    
    traffic = traffic.reset_index()
    
    if granularity_km_ex_post is not None:
        traffic['km'] = traffic.km.apply(lambda x: rounder(x, prec=2, base=granularity_km_ex_post))
    if granularity_hour_ex_post is not None:
        traffic['hour'] = traffic.hour.apply(lambda x: rounder(x, prec=2, base=granularity_hour_ex_post))
    
    traffic = traffic.set_index(['tstamp', 'hour', 'km', 'direction'])[['cars', 'total_weight', 'speed_km_hr',
                                                                       'veh_type_1', 'veh_type_2', 'veh_type_3',
                                                                       'veh_type_4', 'veh_type_5', 'veh_type_6',
                                                                       'veh_type_7', 'speed_sec_km']]

    results = traffic.groupby(['tstamp', 'hour', 'km', 'direction']).sum()
    
    results = results.reset_index()
    # to get the average weight and speed: divide the totals by the amount of cars:
    cols_to_divide = ['total_weight', 'speed_km_hr', 'speed_sec_km']
    for col in cols_to_divide:
        results[col] = results[col] / results['cars']

    # compute ratio of each vehicle type:
    cols_to_sum = [f'veh_type_{i}' for i in range(1,8)]
    results['veh_type_total'] = results[cols_to_sum].sum(axis=1)
    for col in cols_to_sum:     
        results[col] = results[col] / results['veh_type_total']
    results = results.drop(columns=['veh_type_total'])
    return results

## load and preprocess

In [2]:
%%time
# load data:
traffic_final = pd.read_parquet(data_path + traffic_path + traffic_file, engine='pyarrow')
time_segment = pd.read_parquet(data_path + segment_time_file, engine='pyarrow')

# preprocess
# formatting:
traffic_final.ent_occur_time = pd.to_datetime(traffic_final.ent_occur_time)
traffic_final.trans_occur_time = pd.to_datetime(traffic_final.trans_occur_time)

# dropping rows where exit_km = entrance_km (0.01% of the data)
traffic_final = traffic_final[traffic_final.entrance_km != traffic_final.exit_km]

# drop timestamp NAs:
traffic_final = traffic_final.dropna(subset=['ent_occur_time', 'trans_occur_time'])
traffic_final = traffic_final.reset_index(drop=True)

CPU times: user 4min 45s, sys: 8min 30s, total: 13min 16s
Wall time: 28min 23s


In [3]:
%%time
# consider only the relevant variables:
traffic_final = traffic_final[['ent_occur_time', 'trans_occur_time', 'entrance_km', 
                               'exit_km', 'direction', 'total_weight', 'veh_type', 'speed_km_hr']]

# kick out rows where exit time is before entrance time or less then 1 min after:
index_to_keep = [i for i, x in enumerate(traffic_final.trans_occur_time-traffic_final.ent_occur_time) if x.total_seconds() > 60]
traffic_final = traffic_final.loc[index_to_keep, :]
traffic_final = traffic_final.reset_index(drop=True)

CPU times: user 4min 26s, sys: 14.7 s, total: 4min 40s
Wall time: 4min 53s


In [4]:
%%time
# preliminary aggregation for efficiency purposes:
traffic_final['ent_date'] = traffic_final.ent_occur_time.dt.date
traffic_final['ent_hour'] = traffic_final.ent_occur_time.dt.hour
traffic_final['exit_date'] = traffic_final.trans_occur_time.dt.date
traffic_final['exit_hour'] = traffic_final.trans_occur_time.dt.hour
traffic_final['cars'] = 1

# round to choice of segment & time granularity level before aggregation:
if granularity_km_ex_ante is not None:
    traffic_final['entrance_km'] = traffic_final.entrance_km.apply(lambda x: rounder(x, prec=2, base=granularity_km_ex_ante))
    traffic_final['exit_km'] = traffic_final.exit_km.apply(lambda x: rounder(x, prec=2, base=granularity_km_ex_ante))
    
if granularity_hour_ex_ante is not None:
    traffic_final['ent_hour'] = traffic_final.ent_hour.apply(lambda x: rounder(x, prec=2, base=granularity_hour_ex_ante))
    traffic_final['exit_hour'] = traffic_final.exit_hour.apply(lambda x: rounder(x, prec=2, base=granularity_hour_ex_ante)) 

CPU times: user 22.7 s, sys: 3.44 s, total: 26.2 s
Wall time: 29.6 s


## ex ante aggregation

In [5]:
%%time

# vehicle type aggregation:
veh_type_pct = traffic_final.groupby(['ent_date', 'ent_hour', 'exit_date', 'exit_hour', 'entrance_km', 
                                      'exit_km', 'direction'
                      ])['veh_type'].value_counts(dropna=False)
veh_type_pct = veh_type_pct.unstack(fill_value=0)
veh_type_pct = veh_type_pct.rename(columns={i: f'veh_type_{i}' for i in range(1,8)})

# cars, total_weight and speed aggregation: 
# First by summing, later (when assigned to kilometer and time) total_weight and speed will be divided 
# by total amount of cars.
traffic_final = traffic_final.groupby(['ent_date', 'ent_hour', 'exit_date', 'exit_hour', 'entrance_km', 
                                       'exit_km', 'direction'
                      ])['cars', 'total_weight', 'speed_km_hr'].sum() # might fail!

# merge the two:
traffic_final = traffic_final.merge(veh_type_pct, how='left', left_index=True, right_index=True, validate='one_to_one')

traffic_final = traffic_final.reset_index()

# combine hours and date:
traffic_final['ent_occur_time'] = pd.to_datetime(traffic_final['ent_date']) + pd.to_timedelta(traffic_final['ent_hour'], unit='hr')
traffic_final['trans_occur_time'] = pd.to_datetime(traffic_final['exit_date']) + pd.to_timedelta(traffic_final['exit_hour'], unit='hr')

# adding 30 mins to all dates to keep aggregation more accurate 
# (exception: when both timestamps are equal, ent_time only moved 15 mins):
traffic_final['ent_occur_time'] = traffic_final.apply(lambda x: x['ent_occur_time'] + timedelta(minutes=15) if x['trans_occur_time']==x['ent_occur_time'] else x['ent_occur_time'] + timedelta(minutes=30), axis=1)
traffic_final['trans_occur_time'] = traffic_final.trans_occur_time.apply(lambda x: x + timedelta(minutes=30))

# testing whether it worked:
assert len(traffic_final[traffic_final.trans_occur_time==traffic_final.ent_occur_time])==0

# drop redundant columns:
traffic_final = traffic_final.drop(['ent_date', 'ent_hour', 'exit_date', 'exit_hour'], axis=1)

CPU times: user 3min 34s, sys: 52 s, total: 4min 26s
Wall time: 4min 57s


In [ ]:
traffic

## Run main:

In [48]:
%%time
# run the main function:
oversampling_factor = 2
result = main(traffic_final, frac=1/oversampling_factor)

result.to_parquet(data_path + 'Time Segment Data/traffic_feature.pq')
result

0.0% done, 0.0 seconds
0.19% done, 126.99 seconds
0.39% done, 89.65 seconds
0.58% done, 87.01 seconds
0.77% done, 71.99 seconds
0.96% done, 71.73 seconds
1.16% done, 66.41 seconds
1.35% done, 69.94 seconds
1.54% done, 68.28 seconds
1.73% done, 66.58 seconds
1.93% done, 67.92 seconds
2.12% done, 63.43 seconds
2.31% done, 2065.86 seconds
2.5% done, 1928.87 seconds
2.7% done, 88.03 seconds
2.89% done, 92.79 seconds
3.08% done, 103.96 seconds
3.28% done, 109.45 seconds
3.47% done, 99.22 seconds
3.66% done, 112.02 seconds
3.85% done, 102.42 seconds
4.05% done, 83.46 seconds
4.24% done, 94.38 seconds
4.43% done, 89.77 seconds
4.62% done, 81.41 seconds
4.82% done, 91.19 seconds
5.01% done, 85.81 seconds
5.2% done, 88.03 seconds
5.39% done, 84.42 seconds
5.59% done, 100.42 seconds
5.78% done, 87.13 seconds
5.97% done, 89.75 seconds
6.17% done, 80.1 seconds
6.36% done, 84.24 seconds
6.55% done, 101.38 seconds
6.74% done, 101.86 seconds
6.94% done, 109.28 seconds
7.13% done, 116.64 seconds
7.32%

59.15% done, 66.02 seconds
59.34% done, 67.15 seconds
59.53% done, 66.36 seconds
59.72% done, 69.55 seconds
59.92% done, 66.18 seconds
60.11% done, 65.72 seconds
60.3% done, 65.77 seconds
60.49% done, 66.23 seconds
60.69% done, 69.73 seconds
60.88% done, 66.48 seconds
61.07% done, 65.43 seconds
61.26% done, 65.47 seconds
61.46% done, 66.0 seconds
61.65% done, 65.65 seconds
61.84% done, 65.66 seconds
62.04% done, 69.71 seconds
62.23% done, 67.39 seconds
62.42% done, 66.03 seconds
62.61% done, 66.23 seconds
62.81% done, 65.51 seconds
63.0% done, 65.98 seconds
63.19% done, 72.76 seconds
63.38% done, 66.58 seconds
63.58% done, 68.91 seconds
63.77% done, 66.66 seconds
63.96% done, 67.07 seconds
64.15% done, 66.84 seconds
64.35% done, 65.75 seconds
64.54% done, 69.43 seconds
64.73% done, 65.25 seconds
64.93% done, 65.9 seconds
65.12% done, 66.0 seconds
65.31% done, 66.34 seconds
65.5% done, 66.55 seconds
65.7% done, 66.45 seconds
65.89% done, 66.39 seconds
66.08% done, 65.94 seconds
66.27% d

,tstamp,hour,km,direction,cars,total_weight,speed_km_hr,veh_type_1,veh_type_2,veh_type_3,veh_type_4,veh_type_5,veh_type_6,veh_type_7,speed_sec_km
0,2009-07-14,15,52,to adama,40,5101.000000,0.000000,0.700000,0.100000,0.050000,0.150000,0.000000,0.000000,0.000000,5.968006e+05
1,2009-07-14,16,52,to adama,54,2438.518519,0.000000,0.888889,0.074074,0.037037,0.000000,0.000000,0.000000,0.000000,4.598650e+05
2,2009-07-14,16,52,to addis,2,9700.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.369440e+07
3,2009-07-14,17,52,to adama,130,2862.461538,0.000000,0.815385,0.092308,0.061538,0.015385,0.015385,0.000000,0.000000,1.713582e+05
4,2009-07-14,18,52,to adama,108,2817.037037,0.000000,0.796296,0.148148,0.018519,0.037037,0.000000,0.000000,0.000000,3.664364e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5279032,2019-08-31,23,62,to addis,156,5478.205128,88.302065,0.115385,0.102564,0.589744,0.076923,0.025641,0.012821,0.076923,6.076923e+00
5279033,2019-08-31,23,63,to adama,20,2145.000000,111.111183,0.500000,0.200000,0.300000,0.000000,0.000000,0.000000,0.000000,3.140000e+01
5279034,2019-08-31,23,63,to addis,156,5478.205128,88.302065,0.115385,0.102564,0.589744,0.076923,0.025641,0.012821,0.076923,6.076923e+00
5279035,2019-08-31,23,64,to adama,20,2145.000000,111.111183,0.500000,0.200000,0.300000,0.000000,0.000000,0.000000,0.000000,3.140000e+01
